## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import time


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# import spreadsheet
weather_data_df = pd.read_csv("WeatherPy_Database.csv")
weather_data_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Castro,BR,-24.7911,-50.0119,76.33,50,0,6.15,clear sky
1,1,Hirado,JP,33.3597,129.5531,46.00,93,8,4.00,clear sky
2,2,Tupanatinga,BR,-8.7533,-37.3397,95.11,15,10,11.10,clear sky
3,3,Westport,US,41.1415,-73.3579,44.60,39,40,20.71,scattered clouds
4,4,Tiksi,RU,71.6872,128.8694,-25.46,86,100,5.97,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
712,712,Nizhniy Kuranakh,RU,58.8353,125.4869,-21.30,82,89,1.77,overcast clouds
713,713,Kahului,US,20.8947,-156.4700,69.01,72,20,4.61,few clouds
714,714,Gimli,CA,50.6336,-96.9907,-18.40,57,99,13.80,overcast clouds
715,715,Walker,US,30.7335,-95.5669,59.00,77,90,3.44,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Choose your minimum temperature "))
max_temp = float(input("Choose your minimum temperature "))

Choose your minimum temperature 75
Choose your minimum temperature 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
weather_data_df = weather_data_df.loc[weather_data_df['Max Temp'] <= max_temp]
weather_data_df = weather_data_df.loc[weather_data_df['Max Temp'] >= min_temp]
weather_data_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Castro,BR,-24.7911,-50.0119,76.33,50,0,6.15,clear sky
11,11,Barao De Melgaco,BR,-16.1944,-55.9675,89.60,48,75,3.44,broken clouds
14,14,Maragogi,BR,-9.0122,-35.2225,82.89,66,85,12.86,overcast clouds
18,18,Rikitea,PF,-23.1203,-134.9692,77.65,74,100,12.37,overcast clouds
29,29,Butaritari,KI,3.0707,172.7902,81.03,80,89,21.12,light rain
...,...,...,...,...,...,...,...,...,...,...
686,686,Esmeralda,CU,21.8525,-78.1169,89.60,45,40,5.75,scattered clouds
690,690,Eseka,CM,3.6500,10.7667,78.19,78,0,3.69,clear sky
704,704,Mampong,GH,7.0627,-1.4001,83.95,43,57,4.41,broken clouds
708,708,Kayes,ML,14.0000,-11.0000,85.98,13,100,8.19,overcast clouds


In [7]:
# 4a. Determine if there are any empty rows.
weather_data_df.isnull().sum()

City_ID            0
City               0
Country            1
Lat                0
Lng                0
Max Temp           0
Humidity           0
Cloudiness         0
Wind Speed         0
Current Weather    0
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
weather_data_df.dropna(inplace=True)

In [9]:
weather_data_df.isnull().sum()

City_ID            0
City               0
Country            0
Lat                0
Lng                0
Max Temp           0
Humidity           0
Cloudiness         0
Wind Speed         0
Current Weather    0
dtype: int64

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = weather_data_df[["City", "Country", "Max Temp", "Current Weather", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
0,Castro,BR,76.33,clear sky,-24.7911,-50.0119,
11,Barao De Melgaco,BR,89.60,broken clouds,-16.1944,-55.9675,
14,Maragogi,BR,82.89,overcast clouds,-9.0122,-35.2225,
18,Rikitea,PF,77.65,overcast clouds,-23.1203,-134.9692,
29,Butaritari,KI,81.03,light rain,3.0707,172.7902,
31,Atuona,PF,80.11,overcast clouds,-9.8000,-139.0333,
34,Caravelas,BR,83.12,overcast clouds,-17.7125,-39.2481,
39,Saint-Pierre,RE,79.00,broken clouds,-21.3393,55.4781,
42,Bahia Blanca,AR,85.82,clear sky,-38.7196,-62.2724,
43,Bambous Virieux,MU,80.01,scattered clouds,-20.3428,57.7575,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# 6b. Iterate through the hotel DataFrame.
for index, rows in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = rows.Lat
    lng = rows.Lng
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    
    params["location"] = f"{lat},{lng}"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    time.sleep(1.1)
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
hotel_df

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
0,Castro,BR,76.33,clear sky,-24.7911,-50.0119,CHACARA BAILLY
11,Barao De Melgaco,BR,89.60,broken clouds,-16.1944,-55.9675,Tuiuiu Pantanal Hotel
14,Maragogi,BR,82.89,overcast clouds,-9.0122,-35.2225,Salinas do Maragogi All Inclusive Resort
18,Rikitea,PF,77.65,overcast clouds,-23.1203,-134.9692,Pension Maro'i
29,Butaritari,KI,81.03,light rain,3.0707,172.7902,Isles Sunset Lodge
...,...,...,...,...,...,...,...
686,Esmeralda,CU,89.60,scattered clouds,21.8525,-78.1169,Centro de Pioneros Exploradores
690,Eseka,CM,78.19,clear sky,3.6500,10.7667,Complexe Hotelier Ndjiki Mpeck Eseka II
704,Mampong,GH,83.95,broken clouds,7.0627,-1.4001,Adom Upscale Hotel
708,Kayes,ML,85.98,overcast clouds,14.0000,-11.0000,


In [14]:
# Check for null count....null count is 0 so we know that these are empty strings.
hotel_df.isnull().sum()

City               0
Country            0
Max Temp           0
Current Weather    0
Lat                0
Lng                0
Hotel Name         0
dtype: int64

In [15]:
# Convert empty strings into NaN using numpy
import numpy as np
hotel_df['Hotel Name'].replace('',np.nan, inplace=True)

In [16]:
# 16 empties are converted into nulls
hotel_df.isnull().sum()

City                0
Country             0
Max Temp            0
Current Weather     0
Lat                 0
Lng                 0
Hotel Name         15
dtype: int64

In [17]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(inplace=True)
clean_hotel_df = hotel_df

In [18]:
# clean data frame no longer has any nulls or empty cells
clean_hotel_df.isnull().sum()

City               0
Country            0
Max Temp           0
Current Weather    0
Lat                0
Lng                0
Hotel Name         0
dtype: int64

In [19]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))